### Importing The Required Libraries:

In [1]:
import warnings
warnings.filterwarnings('ignore')

import tensorflow as tf
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.text import Tokenizer

tf.__version__
sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(log_device_placement=True))
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)


import pickle
import numpy as np

import os
import os.path

import string
from itertools import chain

Device mapping:
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: NVIDIA GeForce RTX 2080 SUPER, pci bus id: 0000:05:00.0, compute capability: 7.5



In [2]:
filename = "qa-10M"
fname = "pb"
tokenizer = pickle.load(open('tokenizer/tokenizer-' + filename + '.pkl', 'rb'))


In [17]:
file_path = "data/" + fname + "-cleaned.txt"
if(os.path.exists(file_path)):
    data = open("data/" + fname + "-cleaned.txt", "r", encoding = "utf8")
else:
    file = open("data/" + fname + ".txt", "r", encoding = "utf8")
    lines = []
    l = 0
    for i in file:
        lines.append(i)
        l = l + 1
    if (l > 5):
        print("The First Line: ", lines[0])
        print("The Last Line: ", lines[-1])
    else:
        print(len(lines[0]))
    data = ""

    for i in lines:
        data = ' '. join(lines)

    data = data.replace('\n', '').replace('\r', '').replace('\ufeff', '')

    translator = str.maketrans(string.punctuation, ' '*len(string.punctuation)) #map punctuation to space
    data = data.translate(translator)
    data = data.replace('  ', ' ').replace('  ', ' ')

    data = ''.join(filter(lambda x: not x.isdigit(), data))

    with open("data/" + fname + "-cleaned.txt", 'w', encoding='utf-8') as output:
        output.write(data)

data[:360]

956


'Pretty good thanks How about you Yeah I totally agree Capitalism is a fundamentally flawed system that puts profits over people and perpetuates inequality and oppression Right and that s why I believe in the need for a socialist or communist system where the means of production are owned and controlled by the people and resources are allocated based on the n'

In [19]:
z = set(chain(*(line.split() for line in data.split() if line)))
print(len(z))

unique_words = ' '.join(z)
print(unique_words)

112
as interests working willing thanks who and groups totally can Pretty controlled political fundamentally think just but where system than prioritizes needs oppression inequality production Right a educate struggle better How agitate perpetuates are starts we rather or afford that for whole long about not agree profits is s truly to based different fight elite building communist with values socialist up owned allocated flawed their Absolutely give society solidarity over on good it means won all resources by t the one strong social stop hope in be of demand Yeah easy need organize rights I We movement few fighting Capitalism puts build people you Together believe class across why wealthy world The


In [7]:
sequence_data = tokenizer.texts_to_sequences([data])[0]
sequence_data[:10]


[107, 243, 50, 48, 3329, 3, 16, 2284, 1376, 326]

In [8]:
vocab_size = len(tokenizer.word_index) + 1
print(vocab_size)

27784


In [9]:
def defineData():
    sequences = []
    for i in range(1, int(len(sequence_data))):
        words = sequence_data[i-1:i+1]
        sequences.append(words)

    #print("The Length of sequences are: ", len(sequences))
    sequences = np.array(sequences)
    #sequences[:10]


    X = []
    y = []

    for i in sequences:
        X.append(i[0])
        y.append(i[1])

    X = np.array(X)
    y = np.array(y)

    y = to_categorical(y, num_classes=vocab_size)
    #y[:5]
    return X,y

In [10]:
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.models import load_model


model = load_model('nextword-'+filename+'.h5')

checkpoint = ModelCheckpoint("nextword-"+filename+".h5", monitor='loss', verbose=1,
    save_best_only=True, mode='auto')

reduce = ReduceLROnPlateau(monitor='loss', factor=0.2, patience=3, min_lr=0.0001, verbose = 1)

logdir='logsnextword'+filename
tensorboard_Visualization = TensorBoard(log_dir=logdir)

model.compile(loss="categorical_crossentropy", optimizer=Adam(learning_rate=0.001))

In [11]:
X,y = defineData()

print("The Data is: ", X[:5])
print("The responses are: ", y[:5])


The Data is:  [ 107  243   50   48 3329]
The responses are:  [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [12]:
model.fit(X, y, epochs=50, batch_size=64, callbacks=[checkpoint, reduce, tensorboard_Visualization])


Epoch 1/50
3/3 [==============================] - 7s 222ms/step - loss: 11.0130

Epoch 00001: loss improved from inf to 11.01299, saving model to nextword-qa-10M.h5
Epoch 2/50
3/3 [==============================] - 0s 15ms/step - loss: 9.4216

Epoch 00002: loss improved from 11.01299 to 9.42165, saving model to nextword-qa-10M.h5
Epoch 3/50
3/3 [==============================] - 0s 55ms/step - loss: 8.3129

Epoch 00003: loss improved from 9.42165 to 8.31287, saving model to nextword-qa-10M.h5
Epoch 4/50
3/3 [==============================] - 0s 17ms/step - loss: 7.5996

Epoch 00004: loss improved from 8.31287 to 7.59957, saving model to nextword-qa-10M.h5
Epoch 5/50
3/3 [==============================] - 0s 19ms/step - loss: 7.0464

Epoch 00005: loss improved from 7.59957 to 7.04637, saving model to nextword-qa-10M.h5
Epoch 6/50
3/3 [==============================] - 0s 82ms/step - loss: 6.6895

Epoch 00006: loss improved from 7.04637 to 6.68953, saving model to nextword-qa-10M.h5
Epoc

3/3 [==============================] - 0s 66ms/step - loss: 1.6351

Epoch 00050: loss improved from 1.69958 to 1.63505, saving model to nextword-qa-10M.h5


In [13]:
print(X , len(X))

[  107   243    50    48  3329     3    16  2284  1376   326   682   502
  2468 20599  9886  2012    15  4192 21475   599   683    20 13314    20
 20984   507    20    15  4600   597  2284   195   511     5  4286   471
  2468 26118    93  2402  2012  2080     5 21283   539 18860   105  2625
    20  6796  2837     5   683    20 17722   105  8923  3320   442     5
  5514   539  2782   426  2468   396  4096  3175     5 24688   539     5
  1542  2297    16   160  2276  5981  5074  3744    38  2284     6   160
 24923   532  1561  2468  4440 12701   539  1323 15947   683    67   105
 16075   237  2420   471  3507  4105    20 15793  2468  5784   152   127
  4286   237 24186    20    20  5604 17514  5294  4553  4104    20  5835
 21636   168     5 10497   471  2468  5784   152   502  2468   148   477
    38   127    70  5981  9708   237  3092  3917   201    93  2886  3028
   815   127    70  5604  2468  2782    15  5743 15588    20     5  5514
   539    87   683   601   126     5  1542] 163


In [14]:
def Predict_Next_Words(model, tokenizer, text):
    #print(text)
    sequence = tokenizer.texts_to_sequences([text])[0]
    sequence = np.array(sequence)
    res = []
    preds = model.predict(sequence)
    preds1= np.flip(np.argsort(preds,axis=1))
    preds = np.argmax(preds,axis=1)
    for i in range(10):
        res.append((preds1[0][i]))
    #print(len(res))
    predicted_word = ""
    predicted_words = []

    for key, value in tokenizer.word_index.items():
        if value in res:
            predicted_words.append(key)
            #print(key)
            #break

    #print(len(predicted_word))
    return predicted_words

In [21]:

text = "obviously";
export = Predict_Next_Words(model, tokenizer, text)
print(export)

['the', 'that', 'and', 'but', 'we', 'it', 'trump', 'they', 'i', 'reading']


In [11]:
strs = ["All","And","Boy","Book","Call","Car","Chair","Children","City","Dog","Door","Enemy","End","Enough","Eat","Friend","Father","Go","Good","Girl","Food","Hear","House","Inside","Laugh","Listen","Man","Name","Never","Next","New","Noise","Often","Pair","Pick","Play","Room","See","Sell","Sit","Speak","Smile","Sister","Think","Then","Walk","Water","Work","Write","Woman","Yes"]

for i in range(50):
    export = Predict_Next_Words(model, tokenizer, strs[i])
    print(strs[i] + "\t" + "\t".join(export))

All	the	that	it	there	this	i	these	just	of	kinds
And	the	that	they	it	there	so	then	i	we	he
Boy	and	that	you	when	i	odd	yeah	probably	joke	wierd
Book	and	or	there	was	more	together	where	had	we	over
Call	and	the	that	it	to	but	from	him	them	4
Car	and	that	in	they	is	to	was	back	go	i
Chair	that	or	out	so	down	which	with	me	from	of
Children	and	the	it	like	so	on	i	yeah	for	a
City	and	so	materials	oh	from	of	ah	maintenance	truck	wierd
Dog	and	that	is	so	as	with	i	instead	station	thing
Door	and	the	they	it	to	so	but	on	i	yeah
Enemy	and	that	so	on	i	from	who	station	thing	total
End	and	that	or	to	well	i	after	of	ironically	no
Enough	and	the	it	to	but	at	on	i	yeah	a
Eat	and	that	in	us	well	i	just	see	right	a
Friend	that	it	or	out	but	more	at	going	who	of
Father	i	yeah	much	edge	essentially	reasons	wrote	died	speakers	compositions
Go	and	out	to	through	back	on	down	up	ahead	into
Good	and	in	well	glad	point	friend	impression	analogy	job	portion
Girl	what	and	that	in	you	so	well	but	on	of
Food	

In [15]:
print(len(tokenizer.word_index.items()))
for key, value in tokenizer.word_index.items():
    export = Predict_Next_Words(model, tokenizer, key)
    print(str(value) + "\t" + key + "\t" + "\t".join(export))

27783
1	phd	the	that	and	good	is	right	i	a	cool	scientist
2	myspace	that	and	are	would	for	is	with	of	i	a
3	you	like	that	yeah	or	are	just	we	people	i	a
4	lebron	yeah	and	could	just	believe	should	in	won	fight	be
5	the	the	that	and	just	one	is	people	wealthy	a	needs
6	think	the	that	and	it	for	they	of	people	i	about
7	like	you	the	that	and	it	they	in	people	i	a
8	lol	that	yeah	or	are	it	for	is	people	where	i
9	wow	the	and	are	we	it	for	they	in	i	a
10	youtube	that	we	it	of	people	these	where	i	depending	videos
11	nba	the	that	and	or	just	absolutely	is	with	i	a
12	any	and	have	play	world	good	crazy	of	sports	place	support
13	iphones	the	that	and	but	are	would	for	is	i	a
14	imax	that	and	or	are	would	for	is	i	a	puts
15	that	the	and	we	is	i	a	puts	different	s	truly
16	yeah	the	that	and	but	how	are	it	to	for	i
17	amazon	the	that	and	are	we	one	is	right	i	a
18	amazonbasics	the	that	and	we	it	for	of	together	i	a
19	its	the	yeah	and	been	me	why	a	by	an	up
20	and	the	that	and	building	i	build	t

160	it	the	yeah	and	can	was	would	is	won	i	s
161	marvel	the	yeah	and	thanks	good	comics	why	movies	morning	language
162	watch	the	like	and	did	all	it	for	in	a	sunday
163	loved	you	the	that	and	it	to	in	of	school	a
164	hot	that	and	but	as	on	for	is	of	i	a
165	drunk	lol	that	yeah	how	did	are	it	when	of	from
166	jr	that	and	are	would	for	is	with	of	i	a
167	longer	and	but	absolutely	when	for	is	in	with	of	a
168	absolutely	the	that	and	how	are	we	it	for	of	i
169	know	you	the	that	it	they	in	of	people	i	a
170	bet	you	the	that	and	well	he	it	in	i	a
171	library	the	and	but	how	are	it	do	of	i	t
172	aladdin	that	it	absolutely	for	is	people	where	i	s	abstract
173	google	the	that	and	it	for	is	i	sounds	s	abstract
174	mlb	and	to	my	is	teams	chatting	games	chat	game	man
175	zuckerburg	that	and	who	are	would	for	is	i	a	puts
176	none	that	and	but	it	when	of	capitalism	i	t	carolina
177	ram	that	and	are	he	would	for	is	with	i	a
178	golden	think	like	just	know	bet	as	africa	episodes	follow	need
179	losin

318	jerk	the	that	and	he	it	for	is	in	i	a
319	apple	lol	that	yeah	how	are	it	when	for	of	people
320	been	and	pretty	to	great	good	nice	building	a	cool	willing
321	mccartney	that	yeah	and	or	are	is	people	used	i	a
322	wild	you	the	that	he	it	this	so	in	i	a
323	playstation	the	that	and	who	would	for	is	i	a	puts
324	everywhere	you	the	that	did	he	it	this	is	not	i
325	somewhat	and	we	he	it	to	as	for	of	i	a
326	agree	the	how	are	this	with	capitalism	i	t	controlled	oppression
327	hello	the	that	yeah	thanks	yes	are	it	do	i	s
328	white	the	like	and	are	just	good	is	house	i	a
329	donated	the	and	this	his	in	money	ww	games	their	coat
330	likewise	how	are	it	when	if	do	of	why	i	t
331	lot	the	and	but	how	are	it	there	of	i	t
332	id	think	can	say	are	not	re	fight	need	be	educate
333	cs	the	that	yeah	we	for	one	lewis	people	i	earned
334	straight	the	that	yeah	and	just	why	where	a	morning	depp
335	wi	the	that	and	are	is	i	a	puts	different	s
336	romans	think	and	can	just	was	would	do	won	gets	saw
337	p

475	awesome	the	and	how	are	we	it	for	of	i	a
476	never	yeah	and	just	believe	knew	won	had	read	heard	saw
477	one	you	but	did	well	to	as	of	before	a	episode
478	mp	that	and	are	would	for	is	with	where	i	a
479	comics	the	that	yeah	and	we	it	of	i	videos	kidd
480	mc	fu	donalds	crosby	shakur	davis	hammer	frost	actions	emotions	noises
481	dvds	that	and	but	for	is	with	of	i	a	by
482	horrible	that	yeah	are	for	one	is	list	people	where	ways
483	ah	the	wow	that	yeah	and	are	we	ya	for	i
484	my	the	and	thing	horror	life	friend	episodes	mom	spielberg	car
485	probably	the	that	and	agree	for	right	with	not	a	be
486	homepod	the	that	and	who	for	is	with	a	davis	paul
487	go	you	that	and	to	back	on	for	in	a	after
488	twice	the	and	just	been	would	for	in	not	i	a
489	crazy	the	and	but	how	we	it	to	for	i	a
490	hard	the	and	but	to	on	for	with	of	i	a
491	pink	that	yeah	we	as	for	floyd	by	let	funny	need
492	sony	that	and	are	would	for	is	with	i	a	puts
493	much	you	that	and	but	to	they	is	of	i	a
494	fanfest	tha

633	eating	you	the	that	good	now	popcorn	i	a	opportunities	associated
634	eagles	the	yeah	and	watch	start	among	buy	be	thousands	choose
635	heights	that	and	who	or	are	has	for	is	i	a
636	zombieland	the	that	and	or	are	he	for	is	i	a
637	baristas	that	and	or	are	for	is	with	of	i	a
638	baa	that	and	but	for	is	right	with	of	i	a
639	clash	that	and	or	are	for	is	with	of	i	a
640	xmas	and	for	is	right	in	with	of	i	a	by
641	canseco	that	and	but	are	is	right	with	of	i	a
642	insects	the	that	and	who	maybe	for	not	a	adidas	t
643	banjos	the	that	and	he	would	for	is	with	i	a
644	salad	that	and	are	would	for	is	with	where	i	a
645	spectrum	the	that	and	or	it	for	people	system	i	a
646	atmospheric	the	that	yeah	and	streep	cars	take	agent	scattering	swim
647	ironically	the	that	and	would	for	is	where	i	a	s
648	weren	yeah	of	school	capitalism	beyond	build	t	allocated	solidarity	determines
649	spending	and	but	well	we	it	now	they	i	a	hours
650	challa	the	that	and	he	for	is	right	with	i	a
651	naps	that	and	

...

27033	unavoidable	the	that	and	he	would	for	is	with	i	a
27034	outcomes	the	that	and	for	is	of	i	a	by	t
27035	prixe	the	that	and	who	or	has	for	is	i	a
27036	vitamins	that	and	are	he	would	for	is	with	i	a
27037	tender	the	that	and	are	he	is	right	with	i	a
27038	princesses	that	and	are	he	would	for	is	with	i	a
27039	hitter	yeah	and	we	to	as	for	in	of	i	a
27040	frenetic	that	and	but	for	is	with	of	i	a	by
27041	tapping	that	and	or	are	he	would	for	is	i	a
27042	unnecessarily	the	that	and	are	would	is	i	a	puts	s
27043	feats	the	that	and	it	for	is	people	these	i	a
27044	mechanical	that	and	are	would	is	with	of	i	a	by
27045	choppy	the	that	yeah	and	would	for	is	i	a	t
27046	biergarten	that	and	are	he	would	for	is	with	i	a
27047	puzle	that	and	are	would	for	is	with	of	i	a
27048	badass	the	that	and	are	he	would	for	is	i	a
27049	bighourn	that	and	are	for	is	with	of	i	a	by
27050	freelance	that	and	or	are	would	for	is	of	i	a
27051	genealogies	that	and	who	are	would	for	is	i	a	t
27052	disslved	the	lik

27189	lasagna	that	and	are	for	is	with	of	i	a	by
27190	westbrook	the	that	and	who	for	not	a	based	davis	t
27191	void	that	and	but	how	are	it	when	to	for	i
27192	unstoppable	that	and	are	he	would	for	is	with	i	a
27193	retention	the	that	and	who	for	is	i	a	davis	puts
27194	truthful	that	and	or	are	for	is	with	of	i	a
27195	denying	the	that	and	but	it	to	for	in	of	i
27196	negatively	that	and	or	are	would	for	is	with	i	a
27197	defend	the	that	and	who	are	for	is	i	a	puts
27198	unforeseen	that	and	or	it	for	is	of	people	i	a
27199	horizontally	the	that	and	or	are	would	for	is	i	a
27200	remedies	the	that	and	or	he	for	is	with	i	a
27201	pajama	like	that	and	are	would	is	right	with	i	a
27202	hillside	and	but	for	is	in	with	of	i	a	by
27203	disorganization	that	and	are	he	would	is	with	i	a	puts
27204	momentum	the	that	and	it	for	in	there	i	a	up
27205	maury	the	that	and	who	would	for	is	i	a	puts
27206	coral	that	yeah	and	we	it	would	is	where	i	a
27207	memory	that	and	are	is	right	with	people	i	a	hap

27344	meditate	that	and	or	are	would	for	is	people	i	a
27345	polititians	the	that	and	who	has	for	not	a	different	taking
27346	tittles	the	that	and	it	so	one	is	right	i	a
27347	epidsode	that	and	are	he	would	for	is	with	i	a
27348	curryplays	that	and	who	are	would	for	is	with	i	a
27349	birchmere	the	that	and	who	or	are	for	is	i	a
27350	assumptive	like	that	and	we	would	is	i	a	based	t
27351	elect	that	and	are	he	would	is	with	people	i	a
27352	shouted	that	and	are	would	is	right	with	of	i	a
27353	journals	the	that	and	for	is	with	i	a	puts	different
27354	rbin	the	that	and	he	would	for	is	i	a	puts
27355	milano	the	that	and	for	is	in	there	i	a	different
27356	cartoonish	that	and	are	would	is	with	where	i	a	puts
27357	stench	that	and	or	are	would	for	is	of	i	a
27358	naval	that	and	or	are	for	is	with	of	i	a
27359	lalas	the	that	and	or	are	it	for	of	i	a
27360	souffle	the	that	and	who	he	for	is	not	i	a
27361	moneys	that	and	or	are	would	for	is	i	a	puts
27362	canonical	the	that	and	or	he	for	is	

27497	stiff	the	that	yeah	and	we	need	clicking	organizations	truly	earned
27498	darrell	that	and	are	would	for	is	with	of	i	a
27499	miseries	the	that	and	are	would	for	is	i	a	puts
27500	defitnetely	the	that	yeah	and	it	for	is	in	i	a
27501	slits	that	and	are	would	for	is	with	of	i	a
27502	periodic	the	that	and	are	he	would	is	i	a	puts
27503	acrynom	the	that	and	or	it	would	for	is	i	a
27504	scents	the	that	and	it	for	is	right	of	i	a
27505	spill	that	and	are	would	for	is	with	of	i	a
27506	attacked	the	yeah	and	his	them	in	streep	keep	their	her
27507	diane	like	that	and	are	is	right	with	of	i	a
27508	timing	the	that	and	who	would	for	is	i	a	puts
27509	suppress	that	and	are	would	for	is	with	of	i	a
27510	relishing	the	like	that	and	are	for	is	in	i	a
27511	bastard	that	and	are	he	would	for	is	with	i	a
27512	egosurf	that	and	are	he	would	is	right	with	i	a
27513	twains	that	and	are	would	is	right	with	of	i	a
27514	tutu	that	and	or	are	for	is	with	of	i	by
27515	tou	the	and	or	are	for	is	of	i	a	

27653	sensed	the	like	that	and	it	for	is	right	i	a
27654	merely	that	and	are	would	for	is	with	of	a	by
27655	consistantly	the	that	and	for	is	i	a	puts	different	s
27656	pits	that	and	are	he	would	is	with	i	a	puts
27657	reversal	the	that	and	who	are	he	for	is	i	a
27658	invetend	the	that	and	it	in	of	association	i	a	him
27659	chicanery	that	and	are	would	for	is	with	of	i	a
27660	professorship	the	that	and	who	he	for	is	with	i	a
27661	engines	the	that	and	he	it	for	of	people	i	a
27662	indoenesia	the	that	and	for	not	getting	i	a	based	different
27663	intruders	the	that	and	would	for	is	with	where	i	a
27664	regretted	that	and	but	are	is	right	with	of	i	a
27665	bought	the	yeah	years	in	anything	a	an	their	up	million
27666	explainable	and	we	world	good	of	amazing	branches	s	genres	policy
27667	reimagined	the	that	and	are	he	would	for	is	i	a
27668	tangle	that	and	are	would	for	is	with	of	i	a
27669	whooping	the	that	and	or	it	for	is	i	a	different
27670	rodeos	the	that	and	it	would	for	is	with	i